In [4]:
import jax
import optax
import jax.numpy as jnp
import jax.random as random
import random as rand
import time

In [1]:
# params
sequence_length = 40

experiment:
get my posts
input: nothing
output: a post

experiment:
video generator
input: blank transparent canvas
output: the next frame of the video
do a several convolution skip connections on each step (adds changes to each frame)

In [11]:
# process tweet data
import json

# Load the tweets.js file
with open('./tweets.js', 'r', encoding='utf-8') as file:
    # Skip the JavaScript assignment and load only the JSON part
    content = file.read()
    json_data = content.split('=', 1)[1].strip()  # Extract the JSON part after `=`
    json_data = json_data.rstrip(';')  # Remove trailing semicolon if present
    tweets_data = json.loads(json_data)

# Initialize lists for replies and posts
replies = []
posts = []

# Process each tweet in the dataset
for tweet_obj in tweets_data:
    tweet = tweet_obj["tweet"]
    
    if "in_reply_to_status_id_str" in tweet and tweet["in_reply_to_status_id_str"]:
        # It's a reply
        replies.append({
            "id": tweet["id_str"],
            "text": tweet["full_text"],
            "in_reply_to": tweet["in_reply_to_status_id_str"],
            "user": tweet.get("in_reply_to_screen_name", None),
            "created_at": tweet["created_at"]
        })
    else:
        # It's a standalone post
        posts.append({
            "id": tweet["id_str"],
            "text": tweet["full_text"],
            "created_at": tweet["created_at"]
        })

# Output the results
print("Replies:")
for reply in replies[:4]:
    print(reply["text"])

print("\nPosts:")
for post in posts[:4]:
    print(post["text"])


Replies:
@VictorTaelin ive found sonnet 3.5v2 to be surprisingly good at coding. upgraded my tools from v1 to v2 and all of a sudden i have to reprompt it like 1/3rd the time
@dgant &lt;script src="gifeditor.mp3" type="application/json"&gt;
@bozo10n 💪
@sunsettler experimentation games are the best

Posts:
compression always feels so satisfying https://t.co/mM5acJxydL
it only takes one line of code to make a gifboard btw https://t.co/hFlPyNTvRm
RT @calbch: @kuberdenis entrepreneurship is the ultimative vehicle for personal development
just two idiots playing a game of chess https://t.co/USjWySv3W9


In [20]:
# set up and process data
post_limit = -1

stop_char = "🛑"
all_posts = [
  "reply: " + reply["text"] + stop_char for reply in replies[:post_limit]
]
all_posts.extend(
  [
    "post: " + post["text"] + stop_char for post in posts[:post_limit]
  ]
)
rand.shuffle(all_posts)


def get_words(post):
  # break up into alhpanum words, space, and special chars like !?# etc.
  # "what the Dog DOO1in? (idk)" => ['what', ' ', the, ' ', Dog, ' ', 'Doo1in', '?', ' ', '(', 'idk', ')']
  tokens = []
  token = ''
  append_token = False
  for i, c in enumerate(post):
    if i == len(post) - 1:
      append_token = True
    if c.isalnum():
      token += c
    else:
      if token != '': tokens.append(token)
      token = ''
      token += c
      append_token = True
    if append_token:
      append_token = False
      tokens.append(token)
      token = ''
  return tokens

from functools import reduce
vocab = sorted(list(set(
  reduce(
    lambda A, B: A.union(B),
    map(lambda post: set(get_words(post)), all_posts)
))))
vocab_size = len(vocab)
print(f"{vocab_size} tokens")

translations = {
  "encode" : dict([(w, t) for w, t in zip(vocab, range(len(vocab)))]),
  "decode" : dict([(t, w) for w, t in zip(vocab, range(len(vocab)))])
}

token = lambda w: translations["encode"][w] # char to token
word = lambda t: translations["decode"][int(t)] # token to char
encode = lambda ws: jnp.array([token(w) for w in get_words(ws)])
decode = lambda ts: "".join([word(t) for t in ts])


post_token_sets = [encode(post) for post in all_posts]

print(f"loaded {len(post_token_sets)} posts.")


10111 tokens
loaded 3914 posts.


In [21]:
print(post_token_sets[0])

[ 7931   496     1   500   852     1   719     1  5836     1  2462     1
  5236     1  2964     0     0  1273     1  6882     1  7940     1  9126
     1  3233     0     0  1273     1  9141     1  9982     1  3321     1
  8307     1  1165     1  3069     1  5591     1  9982     1  7481     1
  6941     1  6178     1  7109     1  7478     1  9364     1  9985     1
  2811     1  3041     1  9985     1  2946     1  6300 10103]


In [5]:
# train functions
# init rnn params
# inputs ()
def init_rnn_params(key, input_shape, hidden_shape, output_shape):
  keys = random.split(key, 3*sequence_length)
  rnn_params = {
    # xW means shape of W is (input, output)
    # hW and xW should output the shape of h.
    str(n) : {
      "Whh" : random.normal(keys[3*n + 0], shape=(hidden_shape[0], hidden_shape[0])) * jnp.sqrt(2 / hidden_shape[0]),
      "Whx" : random.normal(keys[3*n + 1], shape=(input_shape[0], hidden_shape[0])) * jnp.sqrt(2 / input_shape[0]),
      "Why" : random.normal(keys[3*n + 2], shape=(hidden_shape[0], output_shape[0])) * jnp.sqrt(2 / hidden_shape[0])
    }
    for n in range(sequence_length + 1) # +1 for the empty sequence at the start
  }
  return rnn_params

@jax.jit
def step_start(rnn_params, x):
  hidden_shape = rnn_params[str(0)]["Whh"].shape[0]
  h = jnp.zeros(hidden_shape)
  y, h = step(rnn_params, h, x, n=str(0))
  return y, h

def step(rnn_params, h, x, n):
  z = h @ rnn_params[n]["Whh"] + x @ rnn_params[n]["Whx"]
  h = jax.nn.tanh(z)
  y = h @ rnn_params[n]["Why"]
  return y, h

@jax.jit
def forward(rnn_params, xbow):
  # for now, recreate karpathy's example
  # lma -> mao
  ys = []
  yi, h = step_start(rnn_params, xbow[0]) # try to predict the first token
  h = jax.nn.tanh(h)
  ys.append(yi)
  for n in range(1, len(xbow)):
    y, h = step(rnn_params, h, xbow[n], n=str(n)) # try to predict i+1th token
    ys.append(y)

  return jnp.array(ys)


def init_embedding_params(key, model_dim):
  keys = random.split(key, 10)
  embedding_params = {
    "layer_1" : {
      "w" : random.normal(keys[0], shape=(1, model_dim)),
      "b" : jnp.zeros((model_dim,)),
      }
  }
  return embedding_params

# learned embeddings
"""
@jax.jit
def embed_tokens(embedding_params, tokens):
  # ts[:, None] turns it from [t, t, t, t] to [[t], [t], [t], [t]]. as it should be, a row vector. transpose but for 1d vec.
  x = tokens[:, None] @ embedding_params["layer_1"]["w"] + embedding_params["layer_1"]["b"]
  return x
"""

# one hot embeddings
@jax.jit
def embed_tokens(tokens):
  return jax.nn.one_hot(tokens, vocab_size)

def embed_chars(chars):
  tokens = encode(chars)
  return embed_tokens(tokens)

@jax.jit
def get_loss(rnn_params, xtokens, ytokens):
  xbow = embed_tokens(xtokens)
  logits = forward(rnn_params, xbow)
  ytokens_one_hot = jax.nn.one_hot(ytokens, len(logits[0]))
  cross_entropies = -jnp.sum(jax.nn.log_softmax(logits, axis=-1) * ytokens_one_hot, axis=-1)
  net_cross_entropy_loss = jnp.sum(cross_entropies)
  return net_cross_entropy_loss



In [13]:
# train
keys = random.split(random.PRNGKey(198123), 10)

model_dim = vocab_size # C
input_shape = (model_dim,)
hidden_shape = (64*16,)
output_shape = (vocab_size,) # logits


#embedding_params = init_embedding_params(keys[0], model_dim)

#rnn_params = init_rnn_params(keys[1], input_shape, hidden_shape, output_shape)
#rnn_params.update({"embedding_params" : embedding_params})


scheduler = optax.schedules.linear_onecycle_schedule(
  transition_steps=100_000,
  peak_value=0.003,
  pct_start = 0.3,
  pct_final = 0.9,
  div_factor = 1.25,
  final_div_factor=10,
)
optimizer = optax.chain(
  optax.scale_by_adam(),
  optax.scale_by_schedule(scheduler),
  optax.scale(-1), # params += -learning_rate x grads
)
opt_state = optimizer.init(rnn_params)


@jax.jit
def train_step(rnn_params, xtokens, ytokens, opt_state):
  loss, grads = jax.value_and_grad(get_loss)(rnn_params, xtokens, ytokens)
  param_updates, updated_opt_state = optimizer.update(grads, opt_state, rnn_params)
  updated_params = optax.apply_updates(rnn_params, param_updates)
  return loss, updated_params, updated_opt_state


batches = 100000 # num of post samples to train on
print_every = 1
post_limit = -1
post_indices = random.choice(keys[3], len(post_token_sets[:post_limit]), shape=(batches,), replace=True) # 1 post per batch
print(f'Training on {len(post_indices)} posts')
last_time = time.time()
for batch in range(batches):
  batch_loss = 0
  subsequences = 0
  post_idx = post_indices[batch]
  post_tokens = post_token_sets[post_idx]
  
  max_subsequence = sequence_length
  for pos, _ in enumerate(post_tokens):
    if pos < sequence_length:
      continue # for now. later make it predict the empty sequence
    x_end = pos
    x_start = max(0, pos - sequence_length)
    y_start = x_start + 1
    y_end = x_end + 1
    xtokens = post_tokens[x_start:x_end]
    ytokens = post_tokens[y_start:y_end]
    loss, rnn_params, opt_state = train_step(rnn_params, xtokens, ytokens, opt_state)
    batch_loss += loss
    subsequences += 1
  
  if subsequences == 0: continue
  if batch % print_every == 0:
    batch_time = time.time() - last_time
    last_time = time.time()
    subsequences_per_sec = subsequences/batch_time
    logits = forward(rnn_params, embed_tokens(xtokens))
    yhat = decode(jnp.argmax(logits, axis=-1))
    print(f'batch {batch:4.0f}  loss={batch_loss/subsequences:3.4f}  steps/s={subsequences_per_sec:0.2f}  prediction: "{decode(xtokens)}" => "{yhat}"')

Training on 100000 posts
batch    0  loss=376.5325  steps/s=2.91  prediction: "UR BROBLEM GREEN https://t.co/mstazBvsCM" => "STIISYOORRBBBRMEEEhEENNNt:tt/:///st/B.cz"
batch    1  loss=302.9848  steps/s=98.30  prediction: " is a wild computational rabbithole man." => "tnha si i   wepmmdputaabtlanrabblbt himm"
batch    2  loss=330.2311  steps/s=98.33  prediction: "e algo show you a lot more similar posts" => " stkesdtgg 
haoyyuuu yluom aeressmuar  p"
batch    4  loss=327.4581  steps/s=104.01  prediction: "ouraging way, not stressful for the kid)" => " gngggagng a. ,n   ttttsssstrfrf  r te )"
batch    5  loss=315.5818  steps/s=103.42  prediction: "ding blocks that make it up, recursively" => " thb bblinb c tsbtktkmmtta u,,pe ru vvve"
batch    6  loss=348.8446  steps/s=98.56  prediction: "per solid learning loop, love it love it" => "lyudl𝗼SdSuupS🧠ssuSleararnr“gggp,orpv🍰vv🛑"
batch    7  loss=351.6792  steps/s=104.76  prediction: "em all the cool ML stuff out there that…" => "  he l ll a e 

KeyboardInterrupt: 

In [ ]:
def inference(xchars):
  xbow = embed_chars(xchars)
  logits = forward(rnn_params, xbow)
  yhatbow = jnp.argmax(logits, axis=-1)
  yhatbow_chars = decode(yhatbow)
  return yhatbow_chars

text = 'post: elon is '
print(text, end='')
for i in range(100):
  current_input = text[-sequence_length:] # final $seq_length chars
  next_char = inference(current_input)[-1]
  text += next_char
  print(next_char, end='')